In [35]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import plotly.express as px

In [4]:
df = pd.read_csv('all_data.csv')
df = df.drop('Unnamed: 0', axis=1)
df

,event_date,country,counts,population,events_per_capita,gdp_per_capita,Western,Asia,South America,public_trust_percentage,living_under_2_dollar_percentage
0,2021,Australia,681,25685412,2.651310,60697.245436,1,0,0,51.9,NaN
1,2022,Australia,794,26005540,3.053196,65099.845912,1,0,0,49.9,NaN
2,2020,Austria,354,8916864,3.970006,48789.497850,1,0,0,62.6,0.6
3,2021,Austria,562,8955797,6.275265,53517.890451,1,0,0,61.0,0.4
4,2022,Austria,294,9041851,3.251547,52084.681195,1,0,0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113,2021,United Kingdom,1585,67026292,2.364744,46869.759058,1,0,0,39.5,0.2
114,2022,United Kingdom,1745,66971395,2.605590,46125.255751,1,0,0,39.5,NaN
115,2020,United States,21585,331511512,6.511086,63528.634303,1,0,0,46.5,0.2
116,2021,United States,13147,332031554,3.959563,70219.472454,1,0,0,40.5,0.2


In [5]:
df_scaled = pd.DataFrame()
df_scaled['event_date'] = df['event_date']
df_scaled['country'] = df['country']
df_scaled['Western'] = df['Western']
df_scaled['Asia'] = df['Asia']
df_scaled['South America'] = df['South America']

for feat in df.columns[2:]:
        if feat != 'Western' and feat != 'Asia' and feat != 'South America':
                df_scaled[f'{feat}_scaled'] = ((df[feat] - df[feat].mean()) / df[feat].std()).round(3)
df_scaled

,event_date,country,Western,Asia,South America,counts_scaled,population_scaled,events_per_capita_scaled,gdp_per_capita_scaled,public_trust_percentage_scaled,living_under_2_dollar_percentage_scaled
0,2021,Australia,1,0,0,-0.382,-0.283,-0.834,0.720,0.288,NaN
1,2022,Australia,1,0,0,-0.344,-0.278,-0.699,0.877,0.168,NaN
2,2020,Austria,1,0,0,-0.491,-0.524,-0.391,0.295,0.930,0.095
3,2021,Austria,1,0,0,-0.421,-0.523,0.384,0.464,0.834,-0.154
4,2022,Austria,1,0,0,-0.511,-0.522,-0.632,0.412,0.834,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113,2021,United Kingdom,1,0,0,-0.080,0.311,-0.930,0.226,-0.457,-0.403
114,2022,United Kingdom,1,0,0,-0.026,0.310,-0.849,0.200,-0.457,NaN
115,2020,United States,1,0,0,6.604,4.111,0.463,0.821,-0.036,-0.403
116,2021,United States,1,0,0,3.784,4.118,-0.394,1.060,-0.397,-0.403


In [6]:
df_scaled.corr()

/var/folders/z7/152736054bv0sl_wm9ygspfr0000gn/T/ipykernel_52396/309767592.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_scaled.corr()


,event_date,Western,Asia,South America,counts_scaled,population_scaled,events_per_capita_scaled,gdp_per_capita_scaled,public_trust_percentage_scaled,living_under_2_dollar_percentage_scaled
event_date,1.000000,0.418741,-0.099151,-0.212938,0.019465,-0.062214,0.145310,0.255186,0.230861,-0.058273
Western,0.418741,1.000000,-0.311234,-0.668411,-0.022514,-0.285831,0.235286,0.475514,0.430178,-0.544830
Asia,-0.099151,-0.311234,1.000000,-0.095027,-0.023615,0.245017,-0.280537,-0.012674,-0.105139,NaN
South America,-0.212938,-0.668411,-0.095027,1.000000,0.162801,0.370382,-0.249832,-0.495538,-0.385211,0.723523
counts_scaled,0.019465,-0.022514,-0.023615,0.162801,1.000000,0.813210,0.131153,-0.040161,-0.187741,0.061129
population_scaled,-0.062214,-0.285831,0.245017,0.370382,0.813210,1.000000,-0.246147,-0.147677,-0.273770,0.303462
events_per_capita_scaled,0.145310,0.235286,-0.280537,-0.249832,0.131153,-0.246147,1.000000,0.193235,0.134652,-0.296407
gdp_per_capita_scaled,0.255186,0.475514,-0.012674,-0.495538,-0.040161,-0.147677,0.193235,1.000000,0.710791,-0.462278
public_trust_percentage_scaled,0.230861,0.430178,-0.105139,-0.385211,-0.187741,-0.273770,0.134652,0.710791,1.000000,-0.370835
living_under_2_dollar_percentage_scaled,-0.058273,-0.544830,NaN,0.723523,0.061129,0.303462,-0.296407,-0.462278,-0.370835,1.000000


In [ ]:
filtered_df = df_scaled.dropna(subset=['living_under_2_dollar_percentage_scaled'])
filtered_df

,event_date,country,Western,Asia,South America,counts_scaled,population_scaled,events_per_capita_scaled,gdp_per_capita_scaled,public_trust_percentage_scaled,living_under_2_dollar_percentage_scaled
2,2020,Austria,1,0,0,-0.491,-0.524,-0.391,0.295,0.930,0.095
3,2021,Austria,1,0,0,-0.421,-0.523,0.384,0.464,0.834,-0.154
5,2020,Belgium,1,0,0,-0.347,-0.486,0.559,0.181,-1.057,-0.652
6,2021,Belgium,1,0,0,-0.331,-0.485,0.691,0.404,0.012,-0.652
8,2018,Brazil,0,0,1,1.579,2.368,-0.678,-1.122,-1.819,1.836
...,...,...,...,...,...,...,...,...,...,...,...
109,2020,Switzerland,1,0,0,-0.554,-0.528,-1.083,1.620,2.250,-0.652
112,2020,United Kingdom,1,0,0,0.125,0.312,-0.625,-0.012,-0.745,-0.279
113,2021,United Kingdom,1,0,0,-0.080,0.311,-0.930,0.226,-0.457,-0.403
115,2020,United States,1,0,0,6.604,4.111,0.463,0.821,-0.036,-0.403


In [77]:
def linear_regression(X, y):
    """performs the linear perceptron algorithm which takes in an original X matrix and a y vector
    Args:
        X (2d-array): represents a matrix of a bias column with numeric values representative of the x features 
        y (1d-array): represents a vector of labels (-1 or 1)
    Returns:
        w (1d-array): the final weight vector that determines the direction and orientation of the line of best fit
    """
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y) 

In [80]:
def linreg_predict(Xnew, ynew, m):
    """takes in the X matrix, y vector, and the m vector which contains the coefficients of the calculated line of best fit and outputs a dictionary that contains the predicted y values, the residuals, the mse, and the r2 score

    Args:
        Xnew (1d or 2d-array): which includes all the desired predictor feature values (not including the bias term)
        y (1d-array):  includes all corresponding output values to Xnew
        m (1d-array): contains the coefficients from the line of best fit function
    Returns:
        preds (dictionary): dictionary containing the predicted y values, the residuals between the y values and the predicted y values, the mse, and the r2 score
    """
    linreg_stats = {}
    ypreds = np.matmul(Xnew, m)
    linreg_stats['ypreds'] = ypreds
    resids = ynew - ypreds
    linreg_stats['resids'] = resids
    
    #the mse function
    mse = np.square(resids).mean()
    linreg_stats['mse'] = mse
    linreg_stats['r2'] = r2_score(ynew, ypreds)
    
    return linreg_stats

In [13]:
df_scaled = pd.DataFrame()
df_scaled['event_date'] = df['event_date']
df_scaled['country'] = df['country']
df_scaled['Western'] = df['Western']
df_scaled['Asia'] = df['Asia']
df_scaled['South America'] = df['South America']
df_scaled['counts'] = df['counts']

for feat in df.columns[2:]:
        if feat != 'Western' and feat != 'Asia' and feat != 'South America' and feat != 'counts':
                df_scaled[f'{feat}_scaled'] = ((df[feat] - df[feat].mean()) / df[feat].std()).round(3)
df_scaled

,event_date,country,Western,Asia,South America,counts,population_scaled,events_per_capita_scaled,gdp_per_capita_scaled,public_trust_percentage_scaled,living_under_2_dollar_percentage_scaled
0,2021,Australia,1,0,0,681,-0.283,-0.834,0.720,0.288,NaN
1,2022,Australia,1,0,0,794,-0.278,-0.699,0.877,0.168,NaN
2,2020,Austria,1,0,0,354,-0.524,-0.391,0.295,0.930,0.095
3,2021,Austria,1,0,0,562,-0.523,0.384,0.464,0.834,-0.154
4,2022,Austria,1,0,0,294,-0.522,-0.632,0.412,0.834,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113,2021,United Kingdom,1,0,0,1585,0.311,-0.930,0.226,-0.457,-0.403
114,2022,United Kingdom,1,0,0,1745,0.310,-0.849,0.200,-0.457,NaN
115,2020,United States,1,0,0,21585,4.111,0.463,0.821,-0.036,-0.403
116,2021,United States,1,0,0,13147,4.118,-0.394,1.060,-0.397,-0.403


In [ ]:
# when plotting should i plot the entire data set or just the X_test/y_test ????
plot_df = pd.DataFrame({
    'public_trust_percentage_scaled_3rd': X_test_poly[:, 3], 
    'gdp_per_capita_scaled_3rd': X_test_poly[:, 6],
    'Western': X_test_poly[:, 10],
    'Asia': X_test_poly[:, 11],
    'South_America': X_test_poly[:, 12],
    'public_trust_x_gdp_per_capita': X_test_poly[:, 13],
    'events_per_capita_scaled': y_test
})

# Plot each feature against residuals
for feature in plot_df.columns[:-1]:  # Exclude 'residuals'
    fig = px.scatter(plot_df, x=feature, y='events_per_capita_scaled',
                     title=f"Events per capita vs. {feature} (X feature)")
    fig.show()

In [82]:
# PRESENT MODEL
# Defining my X and y arrays
X = df_scaled[['public_trust_percentage_scaled', 'gdp_per_capita_scaled', 'Western', 'Asia', 'South America']]
y = df_scaled['events_per_capita_scaled']

# Cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Adding Interaction Terms because public trust percentage and gdp per capita are highly correlated  
X_train['public_trust_x_gdp_per_capita'] = X_train['public_trust_percentage_scaled'] * X_train['gdp_per_capita_scaled']
X_test['public_trust_x_gdp_per_capita'] = X_test['public_trust_percentage_scaled'] * X_test['gdp_per_capita_scaled'] 

# addressing multicolinerarity by having these features interact since they are highly correlated
X_train_poly_2a = X_train['public_trust_percentage_scaled'] * X_train['public_trust_percentage_scaled']
X_train_poly_2b = X_train['gdp_per_capita_scaled'] * X_train['gdp_per_capita_scaled']
X_train_poly_3a = X_train['public_trust_percentage_scaled'] * X_train['public_trust_percentage_scaled'] * X_train['public_trust_percentage_scaled']
X_train_poly_3b = X_train['gdp_per_capita_scaled'] * X_train['gdp_per_capita_scaled'] * X_train['gdp_per_capita_scaled']

X_test_poly_2a = X_test['public_trust_percentage_scaled'] * X_test['public_trust_percentage_scaled']
X_test_poly_2b = X_test['gdp_per_capita_scaled'] * X_test['gdp_per_capita_scaled']
X_test_poly_3a = X_test['public_trust_percentage_scaled'] * X_test['public_trust_percentage_scaled'] * X_test['public_trust_percentage_scaled']
X_test_poly_3b = X_test['gdp_per_capita_scaled'] * X_test['gdp_per_capita_scaled'] * X_test['gdp_per_capita_scaled']

# Concatenate features individually to make the model simpler 
# right now my model only has x_a * x_b + x_a^3 * x_a^3 is this what you meant? or should x_a^2 * x_a^2 also be included
X_train_poly = np.concatenate((X_train[['public_trust_percentage_scaled', 'gdp_per_capita_scaled']],
                                 X_train_poly_2a.values.reshape(-1, 1),
                                 X_train_poly_2b.values.reshape(-1, 1),
                                 X_train_poly_3a.values.reshape(-1, 1), 
                                 X_train_poly_3b.values.reshape(-1, 1)), axis=1)
X_test_poly = np.concatenate((X_test[['public_trust_percentage_scaled', 'gdp_per_capita_scaled']],
                                X_test_poly_2a.values.reshape(-1, 1),
                                X_test_poly_2b.values.reshape(-1, 1),
                                X_test_poly_3a.values.reshape(-1, 1), 
                                X_test_poly_3b.values.reshape(-1, 1)), axis=1)

# adding bias column to my X_train and X_test matrix
X_train_poly = np.concatenate((np.ones((X_train_poly.shape[0], 1)), X_train_poly), axis=1)
X_test_poly = np.concatenate((np.ones((X_test_poly.shape[0], 1)), X_test_poly), axis=1)

# Concatenate the transformed features with the original categorical features
X_train_poly = np.concatenate((X_train_poly, X_train[['Western', 'Asia', 'South America', 'public_trust_x_gdp_per_capita']]), axis=1)
X_test_poly = np.concatenate((X_test_poly, X_test[['Western', 'Asia', 'South America', 'public_trust_x_gdp_per_capita']]), axis=1)

# --- Create and Fit Model ---
lobf = linear_regression(X_train_poly, y_train)
predictions = linreg_predict(X_test_poly, y_test, lobf)

# --- Predict and Evaluate ---
mse = mean_squared_error(y_test, predictions['ypreds'])
r2 = r2_score(y_test, predictions['ypreds'])
print("mse:", predictions['mse'], ",r2:", predictions['r2'])

mse: 0.8915582798012633 , r2: 0.32591114608593286


In [ ]:
# Training on the entire data set to prove that serious overfitting is not occuring
X = df_scaled[['public_trust_percentage_scaled', 'gdp_per_capita_scaled', 'Western', 'Asia', 'South America']]
y = df_scaled['events_per_capita_scaled']

# Train-Test Split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# # --- Optionally: Adding Interaction Terms --- 
# # (Only add these if you think the interaction might be important)
X['public_trust_x_gdp_per_capita'] = X['public_trust_percentage_scaled'] * X['gdp_per_capita_scaled']
# Do the same for X_test
#X_test['public_trust_x_gdp_per_capita'] = X_test['public_trust_percentage_scaled'] * X_test['gdp_per_capita_scaled']


# --- #Polynomial Features ---
poly_features = PolynomialFeatures(degree=3)  # Experiment with different degrees
X_poly = poly_features.fit_transform(X[['public_trust_percentage_scaled', 'gdp_per_capita_scaled']])  # Apply only to public_trust
#X_test_poly = poly_features.transform(X_test[['public_trust_percentage_scaled', 'gdp_per_capita_scaled']])

# Concatenate the transformed features with the original categorical features
X_poly = np.concatenate((X_poly, X[['Western', 'Asia', 'South America']]), axis=1)
#X_test_poly = np.concatenate((X_test_poly, X_test[['Western', 'Asia', 'South America']]), axis=1)

# --- Create and Fit Model ---
model = LinearRegression()
model.fit(X_poly, y)

# --- Predict and Evaluate ---
y_pred = model.predict(X_poly)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)
print(mse, r2)

0.7165094597743941 0.2773361379695043


/var/folders/z7/152736054bv0sl_wm9ygspfr0000gn/T/ipykernel_12862/1899431866.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['public_trust_x_gdp_per_capita'] = X['public_trust_percentage_scaled'] * X['gdp_per_capita_scaled']


In [73]:
residuals = y_test - y_pred

residual_df = pd.DataFrame({'residuals': residuals, 'index': range(len(residuals))})

fig = px.scatter(residual_df, x='index', y='residuals',
                 title="Residuals vs. Index")

fig.show()

In [74]:
plot_df = pd.DataFrame({
    'public_trust_percentage_scaled_3rd': X_test_poly[:, 3], 
    'gdp_per_capita_scaled_3rd': X_test_poly[:, 6],
    'Western': X_test_poly[:, 7],
    'Asia': X_test_poly[:, 8],
    'South_America': X_test_poly[:, 9],
    'public_trust_x_gdp_per_capita': X_test_poly[:, 10],
    'residuals': residuals
})

# Plot each feature against residuals
for feature in plot_df.columns[:-1]:  # Exclude 'residuals'
    fig = px.scatter(plot_df, x=feature, y='residuals',
                     title=f"Residuals vs. {feature} (X feature)")
    fig.show()

## Residuals vs Public Trust ^3 Graph
notes on ipad, discuss with gerber and melody tomorrow